Привет! Этот ноутбук -- Ваше домашнее задание к лекции "Статистика: статистики и оценивание". Основная цель данного ноутбука -- продолжить освоение материала и техник, пройденных на лекции. Условия задач старался сделать интересными и приближёнными к реальности (почти!!!). Вас ждёт две задачи на суммарную стоимость 10 баллов. В каждой задаче есть две части -- теоретическая и практическая. Эти две части оцениваются независимо. Т.к. практическая часть опирается на теоретическую, во второй будет приведён "желаемый" ответ, к которому нужно прийти по ходу решения.

В теоретических частях вместо троеточия $\ldots$ вставляйте свои ответы.

Для практической части на основании Ваших ФИО будет вычислен персональный код, с помощью которого будут генерироваться выборки для задач. Везде, где прописано "НЕПРИКОСАЕМЫЙ БЛОК" -- пожалуйста, не трогайте код в этих ячейках.

Если происходят любые проблемы с персональным кодом, используйте вот этот -- '295930404'


По любым вопросам днём и ночью пишите в телегу @shrpv_denis

In [1]:
### Необходимые библиотеки ###

import numpy as np

from zlib import crc32
from scipy.stats import poisson
from scipy.stats import norm
from scipy.stats import binom

In [3]:
### Используйте при необходимости установки ###

# !pip install numpy
# !pip install scipy

In [2]:
### Ваше ФИО на русском языке в формате "Фамилия_Имя_Отчество"
personal_string = 'Майко_Максим_Витальевич'

In [4]:
### НЕПРИКОСАЕМЫЙ БЛОК КОНСТАНТ -- НЕ ТРОГАЙТЕ ЕГО, ПОЖАЛУЙСТА ###
PERSONAL_CODE = crc32(personal_string.encode('utf-8'))
RANDOM_STATE = 42

## Задача 1 (5 баллов)

### Условие
---

Однажды у брокера Васи появилось желание оценивать свой заработок за списание брокерской комиссии. Он поставил следующее бизнес-требование. Для некоторой ценной бумаги необходимо оценить интенсивность сделок с её участием. Для этого была собрана команда молодых аналитиков и выдвинуто предположение, что число сделок в час с этой ценной бумагой имеет распределение Пуассона с параметром $\lambda$ (интенсивность сделок/час). В распоряжении аналитиков есть информация о числе сделок с этой ценной бумагой за каждый рабочий час биржи в течении последних ~3 лет.

Помогите коллегам оценить параметр интенсивности $\lambda$, если Ваша выборка (число сделок в рабочий час) имеет объём

- 1000 дней по 10 рабочих часов
- 500 дней по 10 рабочих часов
- 100 дней по 10 рабочих часов
- 50 дней по 10 рабочих часов
- 10 дней по 10 рабочих часов

### Литература
---

1. Про брокера и его комиссии можно узнать [по ссылке](https://www.tinkoff.ru/invest/education/courses/119543f7-88f7-4611-9038-dfb3e11fafc5/)

### Решение
---

#### (2 балла) 1. Построим оценку максимального правдоподобия параметра $\lambda$ для распределения Пуассона
$$ \mathbb{P}\left\{ X_j = k_j \right\} = \frac{\lambda^k_j e^{-\lambda}}{k_j!},  $$
где
- $k_j$ -- число сделок за $j$-й час
- $\lambda$ -- интенсивность сделок == [сделок / час]
- $\mathbf{X} = (X_1, \ldots, X_n)$ -- простая выборка объёма $n$, $X_j \in \mathrm{Poisson}(\lambda)$.
- $\mathbf{k} = (k_1, \ldots, k_n)$ -- реализация выборки

Функция правдоподобия

$$ L(\lambda) = \frac{\lambda^{\sum_j k_j}}{\prod_j k_j!}\exp{(-n\lambda)} $$

Логарифм функции правдободобия

$$ \ln{L\left( \lambda \right)} = {\sum_j k_j}\ln{\lambda}-{\sum_j \ln (k_j!)}-{n\lambda}  $$

Задача на оптимизацию

$$ \ln{L\left( \lambda \right)} \rightarrow \max_\lambda $$

Первая производная по параметру $\lambda$

$$ \frac{\partial \ln{L\left( \lambda \right)}}{\partial \lambda} = \frac{\sum_j k_j}{\lambda} -{n} $$

Определение знака второй производной по параметру $\lambda$

$$ \frac{\partial^2 \ln{L\left( \lambda \right)}}{\partial \lambda^2} = -\frac{\sum_j k_j}{\lambda^2}  < 0$$

Оценка максимального правдоподобия параметра $\lambda$

$$ \hat{\lambda} = \frac{1}{n}\sum\limits_{j=1}^{n}k_j $$


#### (1 балл) 2. Является ли полученная оценка несмещённой? Т.е. верно ли, что

$$ \mathbb{E} \hat{\lambda} = \lambda ?$$

Заменим реализацию выброки $k_j$ на случайные величины $X_j$

$$ \hat{\lambda}\left( \mathbf{X}^{(n)} \right) = \frac{1}{n}\sum\limits_{j=1}^{n}X_j $$

Если $X_j \in \mathrm{Poisson}(\lambda)$, то

$$ \mathbb{E}X_j = \lambda $$

Тогда в силу свойств линейности математического ожидания получим

$$ \mathbb{E}{\hat{\lambda}} = \lambda$$

Т.е. полученная оценка является несмещённой

#### (2 балла) 3. Перейдём к практике

Реализация функции, которая по входной выборке $\mathbf{X}^{(n)}$ строит оценку максимального правдоподобия параметра $\lambda$

In [5]:
def ex1_solution(X: np.ndarray) -> float:
    return np.mean(X)

In [6]:
### НЕПРИКОСАЕМЫЙ БЛОК КОДА ###

def abs_diff(y_pred: float, y_true: float) -> float:
    return abs(y_pred - y_true)

Генерация выборок $\mathbf{X}^{(n)} = (X_1, \ldots, X_n), X_j \in \mathrm{Poisson}(\lambda)$

In [7]:
### НЕПРИКОСАЕМЫЙ БЛОК КОДА ###

LAMBDA = PERSONAL_CODE % 100 if PERSONAL_CODE % 100 != 0 else 10
HOURS = 10    # Число рабочих часов
DAYS = 1000  # Число рабочих дней

In [8]:
### ГЕНЕРАЦИЯ ВЫБОРКИ ###

X = poisson.rvs(LAMBDA, size=[DAYS, HOURS], random_state=RANDOM_STATE)

Подсчёт результатов

In [9]:
### ГЕНЕРАЦИЯ ВЫБОРКИ ###

X500 = poisson.rvs(LAMBDA, size=[500, HOURS], random_state=RANDOM_STATE)
X100 = poisson.rvs(LAMBDA, size=[100, HOURS], random_state=RANDOM_STATE)
X50 = poisson.rvs(LAMBDA, size=[50, HOURS], random_state=RANDOM_STATE)
X10 = poisson.rvs(LAMBDA, size=[10, HOURS], random_state=RANDOM_STATE)

In [10]:
result1 = np.mean([ex1_solution(x) for x in X])
result2 = np.mean([ex1_solution(x) for x in X500])
result3 = np.mean([ex1_solution(x) for x in X100])
result4 = np.mean([ex1_solution(x) for x in X50])
result5 = np.mean([ex1_solution(x) for x in X10])

In [11]:
### ОТВЕТЫ ДЛЯ САМОКОНТРОЛЯ ###

print(f'LAMBDA=\'{LAMBDA}\', DAYS=\'{DAYS}\', HOURS=\'{HOURS}\'')
print(f'N={1000}, lambda=\'{result1:6f}\', abs_diff=\'{abs_diff(result1, LAMBDA):6f}\'')
print(f'N={500}, lambda=\'{result2:6f}\', abs_diff=\'{abs_diff(result2, LAMBDA):6f}\'')
print(f'N={100}, lambda=\'{result3:6f}\', abs_diff=\'{abs_diff(result3, LAMBDA):6f}\'')
print(f'N={50}, lambda=\'{result4:6f}\', abs_diff=\'{abs_diff(result4, LAMBDA):6f}\'')
print(f'N={10}, lambda=\'{result5:6f}\', abs_diff=\'{abs_diff(result5, LAMBDA):6f}\'')

LAMBDA='46', DAYS='1000', HOURS='10'
N=1000, lambda='46.052100', abs_diff='0.052100'
N=500, lambda='46.041000', abs_diff='0.041000'
N=100, lambda='45.745000', abs_diff='0.255000'
N=50, lambda='45.566000', abs_diff='0.434000'
N=10, lambda='46.340000', abs_diff='0.340000'


## Задача 2 (5 баллов)

### Условие
---

Представим, что перед Вами поставлена задача обучения бинарного классификатора. После кропотливого обучения нейросеть получила $r$ верных ответов на тестовой выборке из n семплов. Постройте доверительный интервал частоты верных ответов к общему числу ответов на доверительном уровне 95%.

### Литература
---

1. Интегральная теорема Муавра-Лапласа -- см. [следствие 6.1](https://old.mipt.ru/education/chair/mathematics/study/uchebniki/%D0%9B_%D0%A2%D0%92_%D0%93%D0%BE%D1%80%D1%8F%D0%B9%D0%BD%D0%BE%D0%B2_2022.pdf) на стр. 55

### Решение
---

#### 1. Формализуем задачу.

Пусть

- $\mathbf{X} = (X_1, \ldots, X_n)$ -- простая выборка объёма $n$, $X_j \in \mathrm{Bern}(p)$.
- $X_j = 1$ $\Leftrightarrow$ нейросеть дала верный ответ с вероятностью $p$;
- $X_j = 0$ $\Leftrightarrow$ нейросеть ошиблась с вероятностью $1-p$.
- $r = \sum_j X_j$ -- число верных ответов нейросети

Тогда требуется по реализации выборки построить доверительный интервал параметра $p$ распределения $\mathrm{Bern}(p)$.

#### (3 балла) 2. Небольшое напоминание.

В случае простой выборки получим $r \in \mathrm{Binom}(n, p)$.

По интегральной теореме Муавра-Лапласа при достаточно большом числе испытаний $n$ справедливо распределение случайной величины $Z$

$$ Z = \frac{r - np}{\sqrt{np(1-p)}} = \frac{r/n - p}{\sqrt{\frac{p(1-p)}{n}}} \sim \ldots$$

Тогда доверительный интервал будет иметь вид

$$ \mathbb{P}\left\{-t_{\alpha/2} \leq {Z} \leq t_{\alpha/2} \right\} \approx 1 - \alpha $$

Преобразуем неравенство к виду

$$ \mathbb{P}\left\{ {p} - t_{\alpha/2}\sqrt{\frac{{p}(1-{p})}{n}} \leq \frac{r}{n} \leq {p} + t_{\alpha/2}\sqrt{\frac{{p}(1-{p})}{n}}  \right\} \approx 1 - \alpha $$

Величина $\sqrt{p(1-p)/n}$ имеет смысл стандартной ошибки точечной оценки $\hat{p} = r/n$. Поэтому в предположении достаточно большого числа измерений заменим параметр $p$ его точечной оценкой $\hat{p}$ в выражении ошибки и окончательно получим

$$ \mathbb{P}\left\{ \hat{p} - t_{\alpha/2}\sqrt{\frac{\hat{p}(1-\hat{p})}{n}} \leq p \leq \hat{p} + t_{\alpha/2}\sqrt{\frac{\hat{p}(1-\hat{p})}{n}}  \right\} \approx 1 - \alpha $$

#### (2 балла) 3. Перейдём к практике

In [12]:
from scipy.stats import norm
def ex2_solution(X: np.ndarray, alpha: float = 0.05) -> list:
  n = len(X)
  Xmean = np.mean(X)
  return [Xmean + np.sqrt(Xmean*(1-Xmean)) * norm.ppf(alpha / 2) / np.sqrt(n), Xmean + np.sqrt(Xmean*(1-Xmean)) * norm.ppf(1 - alpha / 2) / np.sqrt(n)]

In [13]:
def len_interval(interval: list) -> float:
    return interval[1] - interval[0]

In [14]:
### НЕПРИКОСАЕМЫЙ БЛОК КОДА ###
mid = 1 / (PERSONAL_CODE % 100) if PERSONAL_CODE % 100 != 0 else 0.75

In [15]:
### НЕПРИКОСАЕМЫЙ БЛОК КОДА ###
alpha = 0.05
N = 100000
P = mid if mid >= 0.5 else 1 - mid

In [16]:
### ГЕНЕРАЦИЯ ВЫБОРКИ ###

Y = binom.rvs(1, P, size=N, random_state=RANDOM_STATE)

In [17]:
Y10000 = binom.rvs(1, P, size=10000, random_state=RANDOM_STATE)
Y1000 = binom.rvs(1, P, size=1000, random_state=RANDOM_STATE)
Y100 = binom.rvs(1, P, size=100, random_state=RANDOM_STATE)
Y10 = binom.rvs(1, P, size=10, random_state=RANDOM_STATE)

In [18]:
result1 = ex2_solution(Y, alpha)
result2 = ex2_solution(Y10000, alpha)
result3 = ex2_solution(Y1000, alpha)
result4 = ex2_solution(Y100, alpha)
result5 = ex2_solution(Y10, alpha)

In [19]:
### ОТВЕТЫ ДЛЯ САМОКОНТРОЛЯ ###

print(f'alpha=\'{alpha}\', N=\'{N}\', P=\'{P}\'')
print(f'N={100000}, interval=\'{result1}\', len_interval=\'{len_interval(result1):6f}\'')
print(f'N={10000}, interval=\'{result2}\', len_interval=\'{len_interval(result2):6f}\'')
print(f'N={1000}, interval=\'{result3}\', len_interval=\'{len_interval(result3):6f}\'')
print(f'N={100}, interval=\'{result4}\', len_interval=\'{len_interval(result4):6f}\'')
print(f'N={10}, interval=\'{result5}\', len_interval=\'{len_interval(result5):6f}\'')

alpha='0.05', N='100000', P='0.9782608695652174'
N=100000, interval='[0.9780704846800065, 0.9798495153199936]', len_interval='0.001779'
N=10000, interval='[0.9782172689124163, 0.9835827310875837]', len_interval='0.005365'
N=1000, interval='[0.9737597557005517, 0.9902402442994482]', len_interval='0.016480'
N=100, interval='[0.9704986045820121, 1.009501395417988]', len_interval='0.039003'
N=10, interval='[1.0, 1.0]', len_interval='0.000000'
